In [1]:
from unsloth import FastLanguageModel                                                                                                                                                                                                                                           
from transformers import AutoTokenizer


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 02-15 13:38:29 [__init__.py:244] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/home/shougan/projects/aip-fredashi/shougan/balance-budget/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annota

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning/models/llama3-1B",
)

==((====))==  Unsloth 2025.10.1: Fast Llama patching. Transformers: 4.56.1. vLLM: 0.9.2+computecanada.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.189 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0. CUDA: 9.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from tuning.utils.utils import chat_template_func

tokenizer = chat_template_func(tokenizer, "llama-3.3")
print(tokenizer.chat_template)

{% if 'role' in messages[0] %}{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 July 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content'] %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>

" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython
" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", 

: 

In [4]:
from datasets import load_from_disk
from tuning.utils.utils import apply_chat_template, apply_chat_template_pt
from tuning.config import DATASETS_DIR

/home/shougan/projects/aip-fredashi/shougan/balance-budget/tuning


## SFT Datasets

In [5]:
# Load SFT datasets
sft_gsm8k = load_from_disk(f"{DATASETS_DIR}/sft-gsm8k")
sft_tuluif = load_from_disk(f"{DATASETS_DIR}/sft-tuluif")

print("=== SFT GSM8K ===")
print(sft_gsm8k)
print(f"\nExample row:\n{sft_gsm8k['train'][0]}")

print("\n=== SFT TuluIF ===")
print(sft_tuluif)
print(f"\nExample row:\n{sft_tuluif['train'][0]}")

=== SFT GSM8K ===
DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 67655
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 200
    })
})

Example row:
{'messages': [{'content': 'You are a helpful assistant who is an expert at solving math problems. Solve the following math problem and return the solution in the following format:\n\nStep 1: <step 1>\nStep 2: <step 2>\n.\n.\nStep n: <step n>\n\n#### <Final numerical answer>', 'role': 'system'}, {'content': 'Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\nAnswer:', 'role': 'user'}, {'content': 'Step 1: How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nStep 2: How many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n\n#### 72', 'role': 'assis

In [6]:
# Apply chat template to SFT datasets
# SFT datasets have a "messages" column with role/content dicts
sft_gsm8k_templated = apply_chat_template(tokenizer, sft_gsm8k)
print("=== Templated SFT GSM8K ===")
print(sft_gsm8k_templated["train"][0]["text"])

Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


Map:   0%|          | 0/67655 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

=== Templated SFT GSM8K ===
<|im_start|>system
You are a helpful assistant who is an expert at solving math problems. Solve the following math problem and return the solution in the following format:

Step 1: <step 1>
Step 2: <step 2>
.
.
Step n: <step n>

#### <Final numerical answer><|im_end|>
<|im_start|>user
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:<|im_end|>
<|im_start|>assistant
Step 1: How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Step 2: How many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.

#### 72<|im_end|>



In [ ]:
print(tokenizer.apply_chat_template(sft_gsm8k["train"][0]["messages"], tokenize = False, add_generation_prompt = False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are a helpful assistant who is an expert at solving math problems. Solve the following math problem and return the solution in the following format:

Step 1: <step 1>
Step 2: <step 2>
.
.
Step n: <step n>

#### <Final numerical answer><|eot_id|><|start_header_id|>user<|end_header_id|>

Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Step 1: How many clips did Natalia sell in May? ** Natalia sold 48/2 = <<48/2=24>>24 clips in May.
Step 2: How many clips did Natalia sell altogether in April and May? ** Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.

#### 72<|eot_id|>


In [11]:
sft_tuluif_templated = apply_chat_template(tokenizer, sft_tuluif)
print("=== Templated SFT TuluIF ===")
print(sft_tuluif_templated["train"][0]["text"])

Map:   0%|          | 0/29780 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

=== Templated SFT TuluIF ===
<|im_start|>system
You are a helpful assistant who is an expert at responding to prompts by carefully following the given instructions<|im_end|>
<|im_start|>user
Provide a detailed analysis of Candace Parker's defensive techniques in her recent games, excluding the words "aggressive" and "blocking", in the format of a sports commentary script.<|im_end|>
<|im_start|>assistant
[Sports Commentary Script]

[Opening Scene: A packed basketball arena, with fans eagerly awaiting the analysis of Candace Parker’s recent performances on the court.]

Commentator 1: Welcome back, basketball fans! Tonight, we're diving into the defensive prowess of Candace Parker, who has been a standout in her recent games. Her ability to impact the game on the defensive end is truly remarkable, and we've got a lot to unpack.

Commentator 2: That's right! Candace has been showcasing some exceptional skills, especially in how she reads the game. Her anticipation is second to none. You ca